In [1]:
import cv2
import mediapipe as mp


In [4]:
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

def human_and_hand_detector():
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5) as pose_tracker, \
         mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5) as hand_tracker:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            pose_results = pose_tracker.process(rgb_frame)
            if pose_results.pose_landmarks:
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                )
                xs = [lm.x for lm in pose_results.pose_landmarks.landmark]
                ys = [lm.y for lm in pose_results.pose_landmarks.landmark]
                h, w, _ = frame.shape
                x_min, x_max = int(min(xs) * w), int(max(xs) * w)
                y_min, y_max = int(min(ys) * h), int(max(ys) * h)
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0,255,0), 2)

            hand_results = hand_tracker.process(rgb_frame)
            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    mp.solutions.drawing_utils.draw_landmarks(
                        frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                        mp.solutions.drawing_utils.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=3),
                        mp.solutions.drawing_utils.DrawingSpec(color=(255,255,0), thickness=2))
            
            cv2.imshow("Person & Hand Pose Estimation", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
if __name__ == "__main__":
    human_and_hand_detector()